In [70]:
import pymongo
import pandas as pd
import numpy as np
import math
from datetime import datetime

from sklearn.linear_model import LinearRegression ## likes, dislikes 대체

# import dask.dataframe as dd ## 대규모 데이터 처리
# import vaex ## 대규모 데이터 처리

import os
from dotenv import load_dotenv

In [71]:
file_path = 'C:/py_src/awake/data/'

In [72]:
# 계정 / 콘텐츠 분석 데이터셋 불러오기
merge_df_users_fin = pd.read_csv('C:/py_src/awake/data/merge_df_users_fin.csv', low_memory=False)
youtube_videos = pd.read_csv(file_path + 'youtube_videos.csv')

## 계정 데이터 분석

### 기초통계

In [74]:
# 기초통계
merge_df_users_fin[merge_df_users_fin.columns[11:-16]].describe()

,views,redViews,UNSUBSCRIBED,SUBSCRIBED,comments,likes,shares,dislikes,estimatedMinutesWatched,averageViewDuration,averageViewPercentage,estimated_revenue,estimated_ad_revenue,estimated_red_partner_revenue,gross_revenue,cpm,playback_based_cpm,subscribers_count,subscribers_gained,subscribers_lost
count,1.056830e+05,1.056830e+05,1.056830e+05,1.056830e+05,105683.000000,1.056830e+05,105683.000000,105683.000000,1.056830e+05,105683.000000,105683.000000,1.056830e+05,1.056830e+05,105683.000000,1.056830e+05,105683.000000,105683.000000,1.056830e+05,105683.000000,105683.000000
mean,7.669704e+04,1.181762e+04,5.780240e+04,8.794060e+03,36.601629,3.968254e+03,122.175610,38.654117,8.683548e+04,2.590939,35.047682,2.877474e+04,1.501653e+04,3248.437925,2.722400e+04,5030.826465,5499.273283,6.988129e+04,125.901574,41.760690
std,9.172899e+05,6.232938e+04,7.977084e+05,1.190782e+05,326.696229,7.623700e+04,1486.124555,462.391495,8.077970e+05,6.506950,55.212625,1.318711e+05,6.781683e+04,13148.516236,1.231992e+05,22890.346092,6556.226151,1.789418e+05,1278.900197,348.199082
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-1.755000e+03,0.000000,-426.000000,0.000000e+00,0.000000,0.000000,-1.127185e+06,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,3.290000e+02,8.700000e+01,9.100000e+01,3.000000e+00,0.000000,2.000000e+00,1.000000,0.000000,5.450000e+02,0.780000,16.680000,1.175040e+02,4.842900e+01,8.373500,8.782450e+01,24.000000,3232.250500,3.530000e+03,1.000000,1.000000
50%,1.844000e+03,4.900000e+02,9.020000e+02,7.600000e+01,1.000000,2.000000e+01,6.000000,1.000000,2.694000e+03,1.580000,28.000000,1.480631e+03,8.320230e+02,132.648000,1.504621e+03,328.000000,5155.512000,1.506600e+04,8.000000,5.000000
75%,1.212200e+04,3.405500e+03,6.649500e+03,8.305000e+02,9.000000,1.620000e+02,33.000000,5.000000,2.045400e+04,2.580000,47.680000,1.108485e+04,5.085199e+03,1021.478500,9.190117e+03,2119.000000,7229.389500,5.203100e+04,43.000000,22.000000
max,4.064898e+07,1.862117e+06,3.707229e+07,4.546611e+06,35931.000000,3.054524e+06,64529.000000,24353.000000,3.330760e+07,120.230000,15049.760000,5.338431e+06,3.082096e+06,388432.785000,5.601879e+06,994860.000000,489556.625000,3.442657e+06,77446.000000,18821.000000


In [75]:
# 잘못된값 처리
merge_df_users_fin['estimated_revenue'] = np.where(merge_df_users_fin['estimated_revenue'] < 0,
                                                   merge_df_users_fin['estimated_ad_revenue'] + merge_df_users_fin['estimated_red_partner_revenue'],
                                                   merge_df_users_fin['estimated_revenue'])

### 상관관계

In [76]:
# 상관관계
users_corr_matrix = merge_df_users_fin[merge_df_users_fin.columns[11:-16]].corr()
users_corr_matrix

,views,redViews,UNSUBSCRIBED,SUBSCRIBED,comments,likes,shares,dislikes,estimatedMinutesWatched,averageViewDuration,averageViewPercentage,estimated_revenue,estimated_ad_revenue,estimated_red_partner_revenue,gross_revenue,cpm,playback_based_cpm,subscribers_count,subscribers_gained,subscribers_lost
views,1.000000,0.887237,0.990442,0.943630,0.513510,0.989553,0.977800,0.988343,0.977856,-0.018855,0.037667,0.707427,0.527626,0.564268,0.528796,0.512862,0.015210,0.476780,0.945025,0.957050
redViews,0.887237,1.000000,0.852712,0.829370,0.505902,0.830601,0.835948,0.870122,0.871829,-0.036296,0.070093,0.746953,0.569576,0.670893,0.570658,0.574717,0.007111,0.567356,0.807065,0.830827
UNSUBSCRIBED,0.990442,0.852712,1.000000,0.936746,0.490312,0.985804,0.972716,0.976795,0.967478,-0.016759,0.044962,0.677054,0.511655,0.537700,0.512832,0.498079,0.015871,0.457118,0.947833,0.953924
SUBSCRIBED,0.943630,0.829370,0.936746,1.000000,0.529463,0.950954,0.921654,0.925751,0.945687,-0.011540,0.037588,0.769096,0.591106,0.603220,0.592372,0.569441,0.028896,0.535358,0.840056,0.905884
comments,0.513510,0.505902,0.490312,0.529463,1.000000,0.500979,0.497708,0.524899,0.503878,-0.019573,0.043904,0.439432,0.325125,0.350781,0.325862,0.323164,0.007777,0.312102,0.504283,0.524476
likes,0.989553,0.830601,0.985804,0.950954,0.500979,1.000000,0.984213,0.977959,0.970689,-0.013458,0.027295,0.680034,0.505728,0.524644,0.506934,0.484658,0.018521,0.441308,0.936573,0.951134
shares,0.977800,0.835948,0.972716,0.921654,0.497708,0.984213,1.000000,0.967875,0.964019,-0.006929,0.028528,0.686037,0.529940,0.542907,0.530864,0.506659,0.024675,0.447388,0.938846,0.945531
dislikes,0.988343,0.870122,0.976795,0.925751,0.524899,0.977959,0.967875,1.000000,0.962861,-0.020414,0.037492,0.692621,0.500523,0.548045,0.501676,0.484593,0.012684,0.452646,0.958289,0.962212
estimatedMinutesWatched,0.977856,0.871829,0.967478,0.945687,0.503878,0.970689,0.964019,0.962861,1.000000,0.082526,0.026288,0.788171,0.657877,0.669697,0.658758,0.630532,0.033969,0.524253,0.912208,0.939512
averageViewDuration,-0.018855,-0.036296,-0.016759,-0.011540,-0.019573,-0.013458,-0.006929,-0.020414,0.082526,1.000000,-0.042992,0.105156,0.146358,0.365298,0.145600,0.099347,0.078650,0.030085,-0.015928,-0.015658


In [77]:
# 버그로 사용된 수치값 대체
merge_df_users_fin['likes'] = np.where(merge_df_users_fin['likes'] < 0, 0, merge_df_users_fin['likes'])
merge_df_users_fin['dislikes'] = np.where(merge_df_users_fin['dislikes'] < 0, 0, merge_df_users_fin['dislikes'])

### 파생변수

In [78]:
# 파생변수1 - 참여도 관련
merge_df_users_fin['like_rate'] = merge_df_users_fin['likes'] / merge_df_users_fin['views'] ## 좋아요 비율 
merge_df_users_fin['comment_rate'] = merge_df_users_fin['comments'] / merge_df_users_fin['views'] ## 댓글 비율
merge_df_users_fin['share_rate'] = merge_df_users_fin['shares'] / merge_df_users_fin['views'] ## 공유 비율  
merge_df_users_fin['dislike_rate'] = merge_df_users_fin['dislikes'] / merge_df_users_fin['views'] ## 싫어요 비율
merge_df_users_fin['total_engage_rate'] = (merge_df_users_fin['likes'] + merge_df_users_fin['comments'] + merge_df_users_fin['shares']) / merge_df_users_fin['views'] ## 총 참여율
merge_df_users_fin['comment_to_like_rate'] = merge_df_users_fin['comments'] / merge_df_users_fin['likes'] ## 댓글/좋아요 비율

In [79]:
# 파생변수2 - 구독자 관련
merge_df_users_fin['subscriber_increase_rate'] = merge_df_users_fin['subscribers_gained'] / merge_df_users_fin['subscribers_count'] ## 구독자 증가율
merge_df_users_fin['subscriber_decrease_rate'] = merge_df_users_fin['subscribers_lost'] / merge_df_users_fin['subscribers_count'] ## 구독자 감소율
merge_df_users_fin['net_subscribers_change'] = merge_df_users_fin['subscribers_gained'] - merge_df_users_fin['subscribers_lost'] ## 구독자 순증가

In [80]:
# 파생변수3 - 수익 관련
merge_df_users_fin['revenue_per_view'] = merge_df_users_fin['estimated_revenue'] / merge_df_users_fin['views'] ## 조회수당 수익
merge_df_users_fin['ad_revenue_rate'] = merge_df_users_fin['estimated_ad_revenue'] / merge_df_users_fin['estimated_revenue'] ## 광고수익비율
merge_df_users_fin['red_revenue_rate'] = merge_df_users_fin['estimated_red_partner_revenue'] / merge_df_users_fin['estimated_revenue'] ## 프리미엄수익비율
merge_df_users_fin['cpm_to_revenue_ratio'] = merge_df_users_fin['cpm'] - merge_df_users_fin['estimated_revenue'] ## cpm 대비 수익
merge_df_users_fin['revenue_per_ad_impression'] = merge_df_users_fin['estimated_revenue'] / merge_df_users_fin['ad_impressions'] ## 광고노출당 수익
merge_df_users_fin['playback_based_cpm_rate'] = merge_df_users_fin['playback_based_cpm'] / merge_df_users_fin['cpm'] ## 재생 기반 수익

In [81]:
# 파생변수4 - 시청 시간 관련
merge_df_users_fin['revenue_per_minute_watched'] = merge_df_users_fin['estimated_revenue'] / merge_df_users_fin['estimatedMinutesWatched'] ## 시청 시간 당 수익
merge_df_users_fin['avg_view_duration_rate'] = merge_df_users_fin['averageViewDuration'] / merge_df_users_fin['averageViewPercentage'] ## 평균 시청 시간 비율
merge_df_users_fin['watched_time_rate'] = merge_df_users_fin['averageViewPercentage'] * merge_df_users_fin['estimatedMinutesWatched'] ## 재생 비율 대비 시청 시간

In [82]:
# 파생변수5 - 구독자 유형 관련
merge_df_users_fin['subscribed_view_time_rate'] = merge_df_users_fin['SUBSCRIBED'] / (merge_df_users_fin['SUBSCRIBED'] + merge_df_users_fin['UNSUBSCRIBED']) ## 구독자 시청 시간 비율
merge_df_users_fin['unsubscribed_view_time_rate'] = merge_df_users_fin['UNSUBSCRIBED'] / (merge_df_users_fin['SUBSCRIBED'] + merge_df_users_fin['UNSUBSCRIBED']) ## 비구독자 시청 시간 비율

In [83]:
# 파생변수6 - 연령 및 성별 관련
female_col = [col_nm for col_nm in merge_df_users_fin.columns if 'female' in col_nm ]
male_col = [col_nm for col_nm in merge_df_users_fin.columns if ('male' in col_nm) and ('female' not in col_nm)]

youth_col = [col_nm for col_nm in merge_df_users_fin.columns if '13-17' in col_nm]
adult_col = [col_nm for col_nm in merge_df_users_fin.columns if ('18-24' in col_nm) or ('25-34' in col_nm) or ('35-44' in col_nm) or ('45-54' in col_nm)]
older_col = [col_nm for col_nm in merge_df_users_fin.columns if ('55-64' in col_nm) or ('65' in col_nm)]

merge_df_users_fin['female_viewer_rate'] = merge_df_users_fin[female_col].sum(axis=1) ## 여성 시청자 비율
merge_df_users_fin['male_viewer_rate'] = merge_df_users_fin[male_col].sum(axis=1) ## 남성 시청자 비율

merge_df_users_fin['youth_viewer_rate'] = merge_df_users_fin[youth_col].sum(axis=1) ## 청소년 시청자 비율
merge_df_users_fin['adult_viewer_rate'] = merge_df_users_fin[adult_col].sum(axis=1) ## 성인 시청자 비율
merge_df_users_fin['older_viewer_rate'] = merge_df_users_fin[older_col].sum(axis=1) ## 노인 시청자 비율

### 시계열 확인
- looker studio 활용

In [96]:
# 'date' 컬럼을 datetime 형식으로 변환
merge_df_users_fin['date'] = pd.to_datetime(merge_df_users_fin['date'])

# 주(week) 컬럼 생성 (ISO 주 번호 기준)
merge_df_users_fin['date_week'] = merge_df_users_fin['date'].dt.isocalendar().week
merge_df_users_fin['date_week'] = merge_df_users_fin['date'].astype(str).str.split('-').str[0] + '-' + merge_df_users_fin['date_week'].astype(str)
merge_df_users_fin['date_week'] = pd.to_datetime(merge_df_users_fin['date_week'] + '-1', format='%Y-%W-%w') ## looker studio 날짜 형식 위해 주마다 월요일 날짜로 변경
merge_df_users_fin['date_week'] = merge_df_users_fin['date_week'].astype(str)

# 월(month) 컬럼 생성
merge_df_users_fin['date_month'] = merge_df_users_fin['date'].dt.month
merge_df_users_fin['date_month'] = merge_df_users_fin['date'].astype(str).str.split('-').str[0] + '-' + merge_df_users_fin['date_month'].fillna(0).astype(int).astype(str)
merge_df_users_fin['date_month'] = np.where(merge_df_users_fin['date_month']=='NaT-0','',merge_df_users_fin['date_month'])

# 분기(quarter) 컬럼 생성
merge_df_users_fin['date_quarter'] = merge_df_users_fin['date'].dt.to_period('Q').dt.quarter
merge_df_users_fin['date_quarter'] = merge_df_users_fin['date'].astype(str).str.split('-').str[0] + '-' + merge_df_users_fin['date_quarter'].fillna(0).astype(str)
merge_df_users_fin['date_quarter'] = np.where(merge_df_users_fin['date_quarter']=='NaT--1','',merge_df_users_fin['date_quarter'])

In [100]:
merge_df_users_looker = merge_df_users_fin[['youtube_user_id','date','date_week','date_month','date_quarter','viewCount','subscriberCount','videoCount',
                                            'views','estimated_revenue','subscribers_count','estimatedMinutesWatched','averageViewDuration',
                                            'total_engage_rate','net_subscribers_change','averageViewPercentage','revenue_per_view',
                                            'female_viewer_rate', 'male_viewer_rate', 'youth_viewer_rate', 'adult_viewer_rate', 'older_viewer_rate']]

In [101]:
# merge_df_users_looker.to_csv('C:/py_src/awake/data/merge_df_users_looker.csv', encoding='utf-8-sig', index=False)

### y값 설정

#### 참여도 관련 지표 - 총 참여율

In [18]:
# 모두 0인 행 0으로 대체
merge_df_users_fin['total_engage_rate'] = merge_df_users_fin['total_engage_rate'].fillna(0)

In [19]:
merge_df_users_fin['total_engage_rate']

0         0.004723
1         0.006195
2         0.012912
3         0.009255
4         0.011152
            ...   
106369    0.000000
106370    0.000000
106371    0.000000
106372    0.000000
106373    0.000000
Name: total_engage_rate, Length: 106374, dtype: float64

#### 구독자 관련 지표 - 구독자 순증가

In [20]:
merge_df_users_fin['net_subscribers_change'].isnull().sum()

np.int64(0)

#### 시청 시간 관련 지표 - 시청유지율

In [21]:
merge_df_users_fin['averageViewPercentage'].isnull().sum()

np.int64(0)

#### 수익 관련 지표 - 조회수당 예상수익

In [22]:
# 모두 0인 행 0으로 대체
merge_df_users_fin['revenue_per_view'] = merge_df_users_fin['revenue_per_view'].fillna(0)

In [23]:
merge_df_users_fin['revenue_per_view']

0         2.168487
1         2.106251
2         2.119750
3         1.857470
4         2.126285
            ...   
106369    0.000000
106370    0.000000
106371    0.000000
106372    0.000000
106373    0.000000
Name: revenue_per_view, Length: 106374, dtype: float64

## 콘텐츠 분석 데이터셋

In [11]:
youtube_videos

,youtube_user_id,video,end_date,views,redViews,comments,likes,dislikes,shares,estimatedMinutesWatched,...,subscribersGained,subscribersLost,monetizedPlaybacks,adImpressions,cardClickRate,cardTeaserClickRate,cardImpressions,cardTeaserImpressions,cardClicks,cardTeaserClicks
0,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-04-23,3,1,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
1,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-04-30,4,0,0,0,0,0,1,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
2,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-01,2,1,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
3,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-02,3,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
4,64467ea09634a10e3709e1ea,--0HSDH6J7o,2023-05-03,1,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8569506,63fb5daa2a0144119186eca8,NaN,2024-01-28,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
8569507,64809e9719c22b644dde6c44,NaN,2024-01-28,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
8569508,64bbf3cd616bd20e30379bf3,NaN,2024-01-28,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
8569509,6508ff021120b40b4427a4fc,NaN,2024-01-28,0,0,0,0,0,0,0,...,0,0,0.0,0.0,0.0,0.0,0,0,0,0
